## <font color=orange> Project

In [1]:
landmarks = ['class']
for val in range(1, 33+1): # 33 landmarks in total
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

## <font color=orange> - Create CVS files

#### - Import Dependencies 

In [3]:
import mediapipe as mp
import cv2
import numpy

import csv
import os
import numpy as np
from matplotlib import pyplot as plt
import time

### - Main Code 

In [18]:
def first_line_CSV_file(path):

    landmarks = ['class']
    for val in range(1, 33+1): # 33 landmarks in total
        landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

    with open(path, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting = csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

In [10]:
def export_landmark(results, action, path):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)

        with open(path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

The following is a function to automatically label videos.


While the video indicated by the path is reproducing we press on the keyboard the keys (the mapping is stored in the variable "labels") corresponding to the poses we see in the video. These poses are our classes that will automatically be stored in the CSV file with the associated set of landmarks extracted when we press the key.

In [11]:
def labeling_video(path_video, labels, path_CSV):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    cap = cv2.VideoCapture(path_video)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            results = pose.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
            k = cv2.waitKey(1)
            for label,asci in labels.items():
                if k == asci:
                    export_landmark(results, label, path_CSV)

            cv2.imshow('Raw Webcam Feed', image)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break

        cap.release()
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)


### <font color=green> - ***Deadlift***

In [ ]:
path_CSV = 'CSV_files/coords_DL_C_new.csv'
path_videos = ['Videos/CorrectDeadlift_45f.mp4','Videos/RollingDeadlift_45f.mp4','Videos/BackDeadlift_45f.mp4']
labels = {"up":117, "down":100, "down_low":108, "down_roll":114, "up_back":98, "up_roll": 103}
    #    "up": u ,  "down": d , "down_low":l,   "down_roll":r,   "up_back":b,  "up_roll":g

first_line_CSV_file(path_CSV) # this line is only execute once

for path_video in path_videos:
    labeling_video(path_video, labels,path_CSV)
    time.sleep(6)

### <font color=green> - ***Squat***

In [ ]:
path_CSV = 'CSV_files/coords_SQ_C_new.csv'
path_videos = ['Videos/CorrectSquat_45f.mp4','Videos/ForwardSquat_45f.mp4','Videos/DeepSquat.mp4']
labels = {"up":117, "down":100, "down_deep":108, "down_forward":102}
    #    "up": u ,  "down": d , "down_deep":l,   "down_forward":f

first_line_CSV_file(path_CSV) # this line is only executed once

for path_video in path_videos:
    labeling_video(path_video, labels,path_CSV)
    time.sleep(6)

### <font color=green> - ***Bench Press***

In [12]:
landmarks = ['class']
for val in range(1, 22+1): # 22 landmarks in total to avoid hip influence in the model 
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
path_CSV = 'CSV_files/coords_BP_C_new.csv'
path_videos = ['Videos/CorrectBench_45f.mp4', 'Videos/TietBench_45f.mp4','Videos/RollBench_45f.mp4']
labels = {"up":117, "down":100, "down_close":108, "up_close":99, "up_roll":114}
    #     "up": u , "down": d , "down_close":l,   "up_close":c , 'up_roll': r

first_line_CSV_file(path_CSV) # this line is only executed once

for path_video in path_videos:
    labeling_video(path_video, labels, path_CSV)
    time.sleep(6)

## <font color=orange> - Train Models

#### - import Dependencies 

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 

from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

### - Main Code

In [20]:
def Create_sample_label_dataset(path_CSV):
    # Create DataFrame
    df = pd.read_csv(path_CSV)

    # sample and label datasets 
    X = df.drop('class', axis=1) # features
    y=df['class'] #target value

    # divide traning and test dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return (X_train, X_test, y_train, y_test)

In [21]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

def Train_Model(X_train, y_train):
    fitted_models = {}
    for algo, pipeline in pipelines.items():
        model = pipeline.fit(X_train, y_train)
        fitted_models[algo] = model
    
    return fitted_models

In [22]:
# TEST ACCURACY MODELS 
def Test_Accuracy(fitted_models, X_test, y_test):
    for algo, model in fitted_models.items():
        yhat = model.predict(X_test)
        print(algo, accuracy_score(y_test.values, yhat),
            precision_score(y_test.values, yhat, average='macro'),
            recall_score(y_test.values, yhat, average='macro'))
            # the precision_score and recall_score functions are called with the average parameter
            # set to 'macro' instead of 'binary'. This means that the precision and recall scores 
            # will be computed for each of the six classes and then averaged to give a single score 
            # for each metric

### <font color=green> - ***Train Deadlift***

In [ ]:
path_CSV = '/Users/danielguarnizo/Desktop/ProjectCV/CSV_files/coords_DL_C.csv'
X_train, X_test, y_train, y_test = Create_sample_label_dataset(path_CSV)
fitted_models = Train_Model(X_train, y_train)


In [80]:
Test_Accuracy(fitted_models, X_test, y_test)

lr 0.9986559139784946 0.9990310077519379 0.9985875706214689
rc 0.9986559139784946 0.9990310077519379 0.9985875706214689
rf 0.9986559139784946 0.9990310077519379 0.9985875706214689
gb 0.9959677419354839 0.9968178356118406 0.9962355012837536


In [81]:
# CHOOSE AND SAVE MODEL
model = fitted_models['rf']
with open('Models/Deadlift_rf.pkl', 'wb') as f:
    pickle.dump(model, f)

### <font color=green> - ***Train Squat***

In [ ]:
path_CSV = '/Users/danielguarnizo/Desktop/ProjectCV/CSV_files/coords_SQ_C.csv'
X_train, X_test, y_train, y_test = Create_sample_label_dataset(path_CSV)
fitted_models = Train_Model(X_train, y_train)

In [13]:
Test_Accuracy(fitted_models, X_test, y_test)

lr 0.9974424552429667 0.9984756097560976 0.9976851851851851
rc 0.9974424552429667 0.9984756097560976 0.9976851851851851
rf 0.9948849104859335 0.996969696969697 0.9953703703703703
gb 0.9948849104859335 0.9961607949412827 0.9937169312169312


In [14]:
# CHOOSE AND SAVE MODEL
model = fitted_models['rf']
with open('Models/Squat_rf.pkl', 'wb') as f:
    pickle.dump(model, f)

### <font color=green> - ***Train Bench Press***

In [ ]:
path_CSV = 'CSV_files/coords_BP_C.csv'
X_train, X_test, y_train, y_test = Create_sample_label_dataset(path_CSV)
fitted_models = Train_Model(X_train, y_train)

In [24]:
Test_Accuracy(fitted_models, X_test, y_test)

lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 0.9981916817359855 0.9961538461538462 0.9975308641975309


In [25]:
# CHOOSE AND SAVE MODEL
model = fitted_models['rf']
with open('Models/Bench_rf.pkl', 'wb') as f:
    pickle.dump(model, f)

## <font color=orange> - Make Predictions 

#### - Import Dependencies 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import mediapipe as mp
import cv2
import numpy as np

### - Main Code

In [2]:
def Make_Predictions(path_model, ups, downs, webcam):
    with open(path_model, 'rb') as f:
        model = pickle.load(f)

        mp_drawing = mp.solutions.drawing_utils
        mp_pose = mp.solutions.pose

        
        cap = cv2.VideoCapture(webcam)
        counter = 0
        current_stage = ''

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            
            while cap.isOpened():
                ret, image = cap.read()
                # Mirrir image to mak easier the read on the window 
                image = cv2.flip(image, 1)

                # Recolor Feed
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make Detections
                results = pose.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
                
                try:
                    row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                    X = pd.DataFrame([row], columns = landmarks[1:])
                    body_language_class = model.predict(X)[0]
                    body_language_prob = model.predict_proba(X)[0]
                    
                    if body_language_class in downs and body_language_prob[body_language_prob.argmax()] >= 0.1:
                        current_stage = body_language_class
                    elif current_stage in downs and body_language_class in ups and body_language_prob[body_language_prob.argmax()] >= 0.1: # NON RIESCO A LEGGERE
                        current_stage = body_language_class
                        counter +=1
                
                    cv2.rectangle(image, (0,0), (600, 120), (245, 117, 16), -1)

                    # DISPLAY CLASS
                    cv2.putText(image, 'CLASS', (190, 24), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 2, cv2.LINE_AA)
                    cv2.putText(image, body_language_class.split(' ')[0], (180,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 4, cv2.LINE_AA)

                    # DISPLAY PROBABILITY
                    cv2.putText(image, 'PROB', (30, 24), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 2, cv2.LINE_AA)
                    cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), (20,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 4, cv2.LINE_AA)

                    # DISPLAY COUNT
                    cv2.putText(image, 'COUNT', (480, 24), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 2,cv2.LINE_AA)
                    cv2.putText(image, str(counter), (470,80), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),4,cv2.LINE_AA)


                except Exception as e:
                    pass

                cv2.imshow('Mediapipe Feed',image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
        cap.release()
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)

### <font color=green> - ***Deadlift Model***

In [ ]:
#labels = {"up":117, "down":100, "down_low":108, "down_roll":114, "up_back":98, "up_roll": 103}
ups = ["up", "up_back", "up_roll"]
downs = ["down", "down_low", "down_roll"]
model_path = "Models/Deadlift_rf.pkl"
webcam = 0
Make_Predictions(model_path, ups, downs, webcam)


### <font color=green> - ***Squat Model***

In [ ]:
# {"up":117, "down":100, "down_deep":108, "down_forward":102}
ups = ["up"]
downs = ["down", "down_deep", "down_forward"]
model_path = "Models/Squat_rf.pkl"
webcam = 1
Make_Predictions(model_path, ups, downs, webcam)

### <font color=green> - ***Bench Press Model***

In [ ]:
# {"up":117, "down":100, "down_close":108, "up_close":99, "up_roll":114}
ups = ["up","up_close", "up_roll"]
downs = ["down", "down_close"]
model_path = "Models/Bench_rf.pkl"
webcam = 1
Make_Predictions(model_path, ups, downs, webcam)